In [ ]:
import pandas as pd

df1 = pd.read_csv('XAPI_donaldtrump_output.csv')
df2 = pd.read_csv('XAPI_elon_output.csv')


In [ ]:
df1['user'] = 'realdonaldtrump'
df2['user'] = 'elonmusk'

combined_df = pd.concat([df1, df2], ignore_index=True)

print(combined_df.head())

               Date                                              Tweet  \
0  2025-04-13 16:00                      What branch does he serve in?   
1  2025-04-14 14:00  It would be nice if American taxpayers weren't...   
2  2025-04-14 19:00                                     Friends 🇺🇸🤝🏼🇸🇻   
3  2025-04-13 05:00                                 🇺🇸 Best Friends 🇺🇸   
4  2025-04-12 20:00  HOLY SH*T 🚨 Italys Prime Minister Giorgia Melo...   

              user  
0  realdonaldtrump  
1  realdonaldtrump  
2  realdonaldtrump  
3  realdonaldtrump  
4  realdonaldtrump  


In [ ]:
combined_df = pd.concat([df1, df2], ignore_index=True)

combined_df.to_csv('combined_output.csv', index=False)


In [ ]:
combined_df

,Date,Tweet,user
0,2025-04-13 16:00,What branch does he serve in?,realdonaldtrump
1,2025-04-14 14:00,It would be nice if American taxpayers weren't...,realdonaldtrump
2,2025-04-14 19:00,Friends 🇺🇸🤝🏼🇸🇻,realdonaldtrump
3,2025-04-13 05:00,🇺🇸 Best Friends 🇺🇸,realdonaldtrump
4,2025-04-12 20:00,HOLY SH*T 🚨 Italys Prime Minister Giorgia Melo...,realdonaldtrump
...,...,...,...
800,2025-04-14 12:00,The same politicians who sold American workers...,elonmusk
801,2025-04-13 19:00,US Government funding way more terrorist organ...,elonmusk
802,2025-04-14 13:00,HEMMER: Will Mahmoud Khalil be deported?@Steph...,elonmusk
803,2025-04-13 20:00,A level of realism you could never achieve wit...,elonmusk


In [ ]:
!pip install tqdm

In [ ]:
import requests
import pandas as pd
import json
import re
from tqdm import tqdm


combined_df["Is_controversial"] = 0

system_prompt = (
    "You are a helpful assistant that classifies a single tweet into 'controversial' (1) "
    "or 'non-controversial' (0). Return ONLY the digit 1 or 0."
)

definition = (
    "Definition of 'controversial': Controversial content refers to any statement, idea, or expression that is likely to "
    "provoke strong disagreement, debate, or emotional reactions. It often involves sensitive, polarizing, or divisive topics, "
    "and typically reflects opinionated, critical, or challenging viewpoints.\n\n"
)

for idx, row in tqdm(combined_df.iterrows(), total=len(combined_df), desc="Classifying"):
    tweet = row["Tweet"]
    user_prompt = (
        definition
        + f"Tweet: \"{tweet}\"\n\n"
        "Please classify this tweet as controversial (1) or non-controversial (0). "
        "Return ONLY the digit 1 or 0."
    )

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user",   "content": user_prompt}
    ]

    val = 0

    try:
        response = requests.post(
            "http://localhost:11434/api/chat",
            json={
                "model": "llama3",
                "messages": messages,
                "options": {"temperature": 0}
            },
            stream=True
        )

        full_response = ""
        for line in response.iter_lines():
            if not line:
                continue
            try:
                part = json.loads(line.decode("utf-8"))
                content = part.get("message", {}).get("content", "")
                full_response += content
            except json.JSONDecodeError:
                continue

        m = re.search(r"\b(1|0)\b", full_response.strip())
        if m:
            val = int(m.group(1))

    except Exception as e:
        print(f"[Warning] failed at index {idx}: {e}")

    combined_df.at[idx, "Is_controversial"] = val

print(combined_df.head())

Classifying: 100%|████████████████████████████| 805/805 [04:02<00:00,  3.32it/s]

               Date                                              Tweet  \
0  2025-04-13 16:00                      What branch does he serve in?   
1  2025-04-14 14:00  It would be nice if American taxpayers weren't...   
2  2025-04-14 19:00                                     Friends 🇺🇸🤝🏼🇸🇻   
3  2025-04-13 05:00                                 🇺🇸 Best Friends 🇺🇸   
4  2025-04-12 20:00  HOLY SH*T 🚨 Italys Prime Minister Giorgia Melo...   

              user  Is_controversial  
0  realdonaldtrump                 0  
1  realdonaldtrump                 1  
2  realdonaldtrump                 0  
3  realdonaldtrump                 0  
4  realdonaldtrump                 1  


In [ ]:
combined_df

,Date,Tweet,user,Is_controversial
0,2025-04-13 16:00,What branch does he serve in?,realdonaldtrump,0
1,2025-04-14 14:00,It would be nice if American taxpayers weren't...,realdonaldtrump,1
2,2025-04-14 19:00,Friends 🇺🇸🤝🏼🇸🇻,realdonaldtrump,0
3,2025-04-13 05:00,🇺🇸 Best Friends 🇺🇸,realdonaldtrump,0
4,2025-04-12 20:00,HOLY SH*T 🚨 Italys Prime Minister Giorgia Melo...,realdonaldtrump,1
...,...,...,...,...
800,2025-04-14 12:00,The same politicians who sold American workers...,elonmusk,1
801,2025-04-13 19:00,US Government funding way more terrorist organ...,elonmusk,1
802,2025-04-14 13:00,HEMMER: Will Mahmoud Khalil be deported?@Steph...,elonmusk,1
803,2025-04-13 20:00,A level of realism you could never achieve wit...,elonmusk,0


In [ ]:
combined_df.to_csv("classified_tweets.csv", index=False)

In [ ]:
import requests
import pandas as pd
import json
import re
from tqdm import tqdm

API_URL = "http://localhost:11434/api/chat"
MODEL    = "llama3"
SYSTEM_PROMPT = (
    "You are a helpful assistant that classifies a single text snippet into 'controversial' (1) "
    "or 'non-controversial' (0). Return ONLY the digit 1 or 0."
)
DEFINITION = (
    "Definition of 'controversial': Controversial content refers to any statement, idea, or expression that is likely to "
    "provoke strong disagreement, debate, or emotional reactions. It often involves sensitive, polarizing, or divisive topics, "
    "and typically reflects opinionated, critical, or challenging viewpoints.\n\n"
)

def classify_df(df: pd.DataFrame, text_col: str) -> pd.DataFrame:
    """
    Adds an 'Is_controversial' column to df by calling the local Ollama endpoint.
    """
    df = df.copy()
    df["Is_controversial"] = 0

    for idx, text in tqdm(df[text_col].items(), total=len(df), desc=f"Classifying {text_col}"):
        prompt = (
            DEFINITION
            + f"{text_col.capitalize()}: \"{text}\"\n\n"
            "Please classify this as controversial (1) or non-controversial (0). Return ONLY the digit."
        )
        messages = [
            {"role": "system",  "content": SYSTEM_PROMPT},
            {"role": "user",    "content": prompt}
        ]

        val = 0
        try:
            resp = requests.post(
                API_URL,
                json={
                    "model": MODEL,
                    "messages": messages,
                    "options": {"temperature": 0}
                },
                stream=True
            )
            full = ""
            for line in resp.iter_lines():
                if not line:
                    continue
                try:
                    part = json.loads(line.decode("utf-8"))
                    full += part.get("message", {}).get("content", "")
                except json.JSONDecodeError:
                    continue

            m = re.search(r"\b([01])\b", full.strip())
            if m:
                val = int(m.group(1))
        except Exception as e:
            print(f"[Warning] failed at index {idx}: {e}")

        df.at[idx, "Is_controversial"] = val

    return df

tweets_df  = pd.read_csv("tweets.csv")   # must contain a column, e.g., "Tweet"

tweets_out  = classify_df(tweets_df,  "Text")

tweets_out.to_csv("tweets_classified.csv",  index=False)

print("Done! Classified tweets → tweets_classified.csv)


SyntaxError: unterminated string literal (detected at line 78) (<ipython-input-1-4132317da500>, line 78)